<a href="https://colab.research.google.com/github/calvinzyl/Projects/blob/main/data_cleaning_code.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth

In [ ]:
from oauth2client.client import GoogleCredentials

In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [ ]:
#merged datasets 1
#https://drive.google.com/file/d/1ngefCHpfiIeRI7boNrTqv7Qvb8x18ee4/view?usp=sharing

fileDownloaded = drive.CreateFile({"id":"1ngefCHpfiIeRI7boNrTqv7Qvb8x18ee4"})

In [ ]:
fileDownloaded.GetContentFile("merge.csv")

In [ ]:
import pandas as pd
data = pd.read_csv('merge.csv', encoding='utf-8')

In [ ]:
data

,Unnamed: 0,Unnamed: 0.1,Unnamed: 0.1.1,ZIP,TSP: Type of Match (Enhanced),Number of Children (Enhanced),Family composition (Enhanced),Gender code - Person 1,Advantage Individual Marital Status - Person 1,Advantage Individual Marital Status Indicator - Person 1,Person Sequence Number - Person 1,Age in 2-year ranges - Person 1 (Enhanced),Target Home Market Value 2.0 Code,Target Home Market Value 2.0 Indicator,NICHES 5.0 Code,NICHES 5.0 Indicator,Advantage Dwelling Type,Advantage Dwelling Type Indicator,Advantage Home Owner,Advantage Home Owner Indicator,Advantage Length of Residence,Advantage Length of Residence Indicator,Advantage Household Age (Enhanced),Advantage Household Age Indicator (Enhanced),Advantage Presence of Children (Enhanced),Advantage Presence of Children Indicator (Enhanced),Advantage Target Income 3.0,Advantage Target Income 3.0 Indicator,Discretionary Spending Income,Target Net Worth 4.0 Code,Target Net Worth 4.0 Indicator,Household Key,psn,",given_name",middle_initial,surname,contracted_address,city,state,zip,...,sc_warehouse_clubs_13-15m_trans,sc_warehouse_clubs_13-15m_spend,sc_warehouse_clubs_16-18m_trans,sc_warehouse_clubs_16-18m_spend,sc_warehouse_clubs_19-21m_trans,sc_warehouse_clubs_19-21m_spend,sc_warehouse_clubs_22-24m_trans,sc_warehouse_clubs_22-24m_spend,sc_wine_and_liquor_0-3m_trans,sc_wine_and_liquor_0-3m_spend,sc_wine_and_liquor_4-6m_trans,sc_wine_and_liquor_4-6m_spend,sc_wine_and_liquor_7-9m_trans,sc_wine_and_liquor_7-9m_spend,sc_wine_and_liquor_10-12m_trans,sc_wine_and_liquor_10-12m_spend,sc_wine_and_liquor_13-15m_trans,sc_wine_and_liquor_13-15m_spend,sc_wine_and_liquor_16-18m_trans,sc_wine_and_liquor_16-18m_spend,sc_wine_and_liquor_19-21m_trans,sc_wine_and_liquor_19-21m_spend,sc_wine_and_liquor_22-24m_trans,sc_wine_and_liquor_22-24m_spend,sc_wireless_0-3m_trans,sc_wireless_0-3m_spend,sc_wireless_4-6m_trans,sc_wireless_4-6m_spend,sc_wireless_7-9m_trans,sc_wireless_7-9m_spend,sc_wireless_10-12m_trans,sc_wireless_10-12m_spend,sc_wireless_13-15m_trans,sc_wireless_13-15m_spend,sc_wireless_16-18m_trans,sc_wireless_16-18m_spend,sc_wireless_19-21m_trans,sc_wireless_19-21m_spend,sc_wireless_22-24m_trans,sc_wireless_22-24m_spend
0,0,164,5358626,22314,A,NaN,6.0,1.0,2.0,H,3300572A0311940000A51A,62.0,890.0,S,N1,S,5.0,S,4.0,S,8.0,S,5.0,S,0.0,H,7,H,4,7,H,3.900337e+09,3300572A0311940000A51A,"3231631,RANDALL",C,BERG,400 MADISON ST APT 801,ALEXANDRIA,VA,22314,...,NaN,NaN,NaN,NaN,2.0,D,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,F,NaN,NaN,NaN,NaN,1.0,8,1.0,9,1.0,8,1.0,3,NaN,NaN
1,1,251,5358713,22314,A,NaN,12.0,1.0,2.0,H,3300572A0311FA019775B8,60.0,1476.0,S,I1,S,1.0,S,4.0,S,5.0,S,5.0,S,0.0,H,C,H,A,A,H,5.089726e+08,3300572A0311FA019775B8,"3254921,JOHN",Q,BINFORD,125 MONTGOMERY PL,ALEXANDRIA,VA,22314,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.0,8,2.0,5,2.0,1,2.0,2,1.0,1,2.0,2,3.0,8,3.0,A,1.0,8,1.0,3,1.0,I,1.0,1,1.0,1,1.0,1,1.0,D,NaN,NaN
2,2,278,5358740,22314,A,NaN,2.0,2.0,1.0,S,330055F3030E0C000B3592,72.0,1240.0,S,C3,S,5.0,S,4.0,S,4.0,S,6.0,S,0.0,H,D,S,9,B,H,2.183194e+09,330055F3030E0C000B3592,"3252778,LESLIE",L,BAER,824 WATER PL,ALEXANDRIA,VA,22314,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3,282,5358744,22314,A,NaN,2.0,2.0,1.0,S,3300572A0310CF0001DD4F,40.0,1313.0,S,D1,S,5.0,S,4.0,S,5.0,S,4.0,S,1.0,H,D,H,C,9,H,4.782153e+07,3300572A0310CF0001DD4F,"3324540,MICHELLE",M,MCDUFFIE,822 WATER PL,ALEXANDRIA,VA,22314,...,2.0,6,NaN,NaN,2.0,A,NaN,NaN,4.0,5,3.0,2,3.0,7,3.0,D,2.0,6,3.0,H,2.0,7,3.0,5,1.0,7,NaN,NaN,NaN,NaN,1.0,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4,374,5358836,22314,A,NaN,12.0,1.0,2.0,S,3300571D0310C7000083BD,42.0,479.0,A,J3,S,5.0,S,NaN,NaN,5.0,S,3.0,S,0.0,H,6,S,4,3,H,3.742277e+09,3300571D0310C7000083BD,"3231203,ERIC",J,LARRONDO,801 N PITT ST APT 1007,ALEXANDRIA,VA,22314,...,3.0,9,NaN,NaN,1.0,2,NaN,NaN,1.0,4,2.0,3,1.0,9,3.0,3,1.0,1,1.0,1,2.0,1,1.0,1,3.0,F,1.0,4,2.0,

In [ ]:
data.filter(regex = 'automotive', axis= 1)

,c_automotive_0-3m_trans,c_automotive_0-3m_spend,c_automotive_4-6m_trans,c_automotive_4-6m_spend,c_automotive_7-9m_trans,c_automotive_7-9m_spend,c_automotive_10-12m_trans,c_automotive_10-12m_spend,c_automotive_13-15m_trans,c_automotive_13-15m_spend,c_automotive_16-18m_trans,c_automotive_16-18m_spend,c_automotive_19-21m_trans,c_automotive_19-21m_spend,c_automotive_22-24m_trans,c_automotive_22-24m_spend
0,3.0,3,6.0,D,4.0,4,3.0,3,3.0,3,2.0,1,2.0,2,1.0,1
1,3.0,4,NaN,NaN,4.0,C,NaN,NaN,3.0,8,NaN,NaN,3.0,3,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,6.0,A,4.0,A,7.0,I,4.0,8,4.0,5,6.0,B,7.0,K,4.0,8
4,3.0,9,3.0,4,NaN,NaN,1.0,1,1.0,3,NaN,NaN,2.0,2,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
150931,1.0,1,4.0,5,1.0,1,4.0,9,1.0,1,3.0,5,1.0,1,4.0,8
150932,2.0,2,1.0,2,2.0,1,3.0,2,NaN,NaN,2.0,1,NaN,NaN,3.0,3
150933,7.0,U,7.0,Q,7.0,R,7.0,Q,7.0,Q,7.0,R,7.0,R,7.0,S
150934,NaN,NaN,1.0,1,NaN,NaN,1.0,1,NaN,NaN,NaN,NaN,1.0,1,NaN,NaN


### Separate and keep geographic features for possible mapping visualization

In [ ]:
geo= data[['city','state','zip','zip_plus4']]

In [ ]:
count = data['Person Sequence Number - Person 1']

generate some variables based on consumptions on categories

clean the merged data

In [ ]:
import seaborn as sns
import pandas as pd
import numpy as np
import os

In [ ]:
#delete indicators that do not contain relevant information
data.filter(regex = 'Indicator$', axis= 1)


,Target Home Market Value 2.0 Indicator,NICHES 5.0 Indicator,Advantage Dwelling Type Indicator,Advantage Home Owner Indicator,Advantage Length of Residence Indicator,Advantage Target Income 3.0 Indicator,Target Net Worth 4.0 Indicator
0,S,S,S,S,S,H,H
1,S,S,S,S,S,H,H
2,S,S,S,S,S,S,H
3,S,S,S,S,S,H,H
4,A,S,S,NaN,S,S,H
...,...,...,...,...,...,...,...
150931,H,S,S,S,S,H,H
150932,H,S,S,S,S,H,H
150933,S,S,S,S,S,H,H
150934,S,S,S,S,S,H,H


In [ ]:
data.drop(columns = data.filter(regex = 'Indicator$', axis= 1), inplace= True)

In [ ]:
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import LabelEncoder
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer

### Recode string indices into ordered numbers in x

In [ ]:
x= data[['Number of Children (Enhanced)','Advantage Target Income 3.0',
         'Discretionary Spending Income','Target Net Worth 4.0 Code',]]

In [ ]:
x['Target Net Worth 4.0 Code'] = x['Target Net Worth 4.0 Code'].replace(['A','B'],[10,11])

In [ ]:
x['Advantage Target Income 3.0'] = x['Advantage Target Income 3.0'].replace(['A','B', 'C', 'D'],[10,11, 12, 13])

In [ ]:
x['Discretionary Spending Income'] = x['Discretionary Spending Income'].replace(['A','B', 'C'],[10,11, 12])

### Using Iterative Imputer to impute missing values in x

In [ ]:
x_clean=pd.DataFrame(IterativeImputer(initial_strategy='median',imputation_order='ascending').fit_transform(x),columns=x.columns)

### Since all five variables in x are supposed to be ordinal integers in nature, so I round the decimals generated from iterative imputer to nearest integers

In [ ]:
x_clean=x_clean.round()

### Using Iterative Imputer to impute missing values in x2

In [ ]:
x2 = data[['Family composition (Enhanced)', 'Advantage Individual Marital Status - Person 1', 
           'Gender code - Person 1','Advantage Dwelling Type','Advantage Home Owner',
           'Advantage Length of Residence','NICHES 5.0 Code']]

In [ ]:
x2['NICHES 5.0 Code'] = x2['NICHES 5.0 Code'].str[:1]
x2['NICHES 5.0 Code'] = x2['NICHES 5.0 Code'].replace(['A','B','C','D','E','F',
                                                     'G','H','I','J','K','L',
                                                     'M','N','O','P','Q','R',
                                                     'S','T','U','V', 'W','X',
                                                     'Y','Z'], [1, 2, 3, 4, 5, 
                                                                6, 7, 8, 9, 10,
                                                                11, 12, 13, 14, 
                                                                15, 16, 17, 18,
                                                                19, 20, 21, 22, 
                                                                23, 24, 25, 26])

In [ ]:
x2_clean=pd.DataFrame(IterativeImputer(initial_strategy='median',imputation_order='ascending').fit_transform(x2),columns=x2.columns)

### Since all six variables in x2 are supposed to be ordinal integers in nature, so I round the decimals generated from iterative imputer to nearest integers

In [ ]:
x2_clean=x2_clean.round()

change marital status to a binary variable with 1 being married and 0 being single, for future classification models possibly

In [ ]:
x2_clean['Advantage Individual Marital Status - Person 1']=x2_clean['Advantage Individual Marital Status - Person 1'].replace([2],[0])

In [ ]:
x2_clean['Gender code - Person 1']=x2_clean['Gender code - Person 1'].replace([2,3],[0,0])

### MANUALLY dummify variables in x2 just for more visualization options

In [ ]:
ouch = x2_clean.copy()

In [ ]:
x_clean['Number of Children (Enhanced)'].value_counts().sort_index(ascending=True)

0.0     3398
1.0    70581
2.0    61847
3.0     9093
4.0     2750
5.0     1732
6.0      935
7.0      402
8.0      150
9.0       48
Name: Number of Children (Enhanced), dtype: int64

In [ ]:
aye = x_clean['Number of Children (Enhanced)']

In [ ]:
ayye=pd.DataFrame(aye)
ayye

,Number of Children (Enhanced)
0,2.0
1,2.0
2,3.0
3,1.0
4,1.0
...,...
150931,2.0
150932,1.0
150933,1.0
150934,2.0


In [ ]:
ouch = pd.concat([ouch, ayye],axis=1)

In [ ]:
ouch['NICHES 5.0 Code (Specified)'] = ouch['NICHES 5.0 Code'].replace([1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 
                                                           15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26],
                                                          ['1: Already Affluent','2: Big Spender Parents','3: Chic Society',
                                                           '4: Doing Well and Donating','5: Easy Street','6: Feathering the Nest',
                                                           '7: Go-Go Families','8: Home Hoppers','9: IRA Spenders','10: Just Sailing Along',
                                                           '11: Kiddie Kastles','12: Loose Change','13: Mid-Life Munchkins',
                                                           '14: Nice and Easy Living','15: Oodles of Offspring','16: Parks, Parts and Playgrounds',
                                                           '17: Quiet Homebodies','18: Resourceful Realists','19: Still Going Strong',
                                                           '20: Tireless and on the Move','21: Undaunted Do-It-Yourselfers','22: Value-Focused',
                                                           '23: Working Hard','24: Xtra Time on Their Hands','25: Young at Heart','26: Zone of Contentment'])

In [ ]:
ouch['Children (categorical)']=ouch['Number of Children (Enhanced)'].replace([0,1,2,3,4,5,6,7,8,9],
                                                                   ['None','One','Two','Three','Four','Five','Six','Seven','Eight','Nine'])

In [ ]:
ouch['Family Composition (categorical)'] = ouch['Family composition (Enhanced)'].replace([1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16],
                                                                                      ['Married, husband and wife present, with children',
                                                                                       'Married, husband and wife present, with no children',
                                                                                       'One male and one female, marital status unknown, with children',
                                                                                       'One male and one female, marital status unknown, with no children',
                                                                                       'Male householder with one or more persons, with children',
                                                                                       'Male householder with one or more persons, with no children',
                                                                                       'Female householder with one or more persons, with children',
                                                                                       'Female householder with one or more persons, with no children',
                                                                                       'Two or more persons of unknown gender with children',
                                                                                       'Two or more persons of unknown gender with no children',
                                                                                       'One person, male householder, with children',
                                                                                       'One person, male householder, with no children',
                                                                                       'One person, female householder, with children',
                                                                                       'One person, female householder, with no children',
                                                                                       'One person, unknown gender, with children',
                                                                                       'One person, unknown gender, with no children'])

In [ ]:
ouch['Marital Status']=ouch['Advantage Individual Marital Status - Person 1'].replace([1,0],['Married','Single'])

In [ ]:
ouch['Gender (categorical)']=ouch['Gender code - Person 1'].replace([1,0],['Male','Female'])

In [ ]:
ouch['Dwelling Type (categorical)']=ouch['Advantage Dwelling Type'].replace([0,1,2,3,5,6],['Unknown','SFDU','MFDU','Business','Condo','Mobile home'])

In [ ]:
ouch['Home Owner (categorical)']=ouch['Advantage Home Owner'].replace([0,1,2,3,4],
                                                                  ['Unknown',
                                                                   'Definite renter',
                                                                   'Probable renter',
                                                                   'Probable owner',
                                                                   'Definite owner'])

In [ ]:
ouch['Length of Residence (categorical)']=ouch['Advantage Length of Residence'].replace([1,2,3,4,5,6,7,8],
                                                                                    ['0-6 months','7-12 months',
                                                                                     '1-2 years','3-5 years',
                                                                                     '6-10 years','11-15 years',
                                                                                     '16-20 years','20+ years'])

In [ ]:
dummies = ouch[['NICHES 5.0 Code (Specified)','Children (categorical)','Family Composition (categorical)', 'Marital Status', 
                'Gender (categorical)','Dwelling Type (categorical)','Home Owner (categorical)','Length of Residence (categorical)']]

In [ ]:
dummies

,NICHES 5.0 Code (Specified),Children (categorical),Family Composition (categorical),Marital Status,Gender (categorical),Dwelling Type (categorical),Home Owner (categorical),Length of Residence (categorical)
0,14: Nice and Easy Living,Two,"Male householder with one or more persons, wit...",Single,Male,Condo,Definite owner,20+ years
1,9: IRA Spenders,Two,"One person, male householder, with no children",Single,Male,SFDU,Definite owner,6-10 years
2,3: Chic Society,Three,"Married, husband and wife present, with no chi...",Married,Female,Condo,Definite owner,3-5 years
3,4: Doing Well and Donating,One,"Married, husband and wife present, with no chi...",Married,Female,Condo,Definite owner,6-10 years
4,10: Just Sailing Along,One,"One person, male householder, with no children",Single,Male,Condo,Definite owner,6-10 years
...,...,...,...,...,...,...,...,...
150931,9: IRA Spenders,Two,"One person, female householder, with no children",Married,Female,Condo,Definite owner,16-20 years
150932,14: Nice and Easy Living,One,"One person, female householder, with no children",Single,Female,Condo,Definite owner,7-12 months
150933,5: Easy Street,One,"Married, husband and wife present, with children",Married,Male,SFDU,Definite owner,16-20 years
150934,2: Big Spender Parents,Two,"Married, husband and wife present, with no chi...",Married,Female,SFDU,Definite owner,6-10 years


### Recode spend codes' indices into average values

In [ ]:
spend = data.filter(regex = 'spend',axis= 1)

In [ ]:
spend1 = spend.replace(['1','2','3','4','5','6','7','8','9','A','B','C','D',
                        'E','F','G','H','I','J','K','L','M','N','O','P','Q','R',
                        'S','T','U','V','W','X','Y','Z']
                       ,[25.50,62.50,88.00,112.50,137.50,162.50,187.50,225.00,
                         275.00,325.00,375.00,430.00,475.00,525.00,575.00,625.00,
                         675.00,725.00,775.00,825.00,875.00,925.00,975.00,1125.00,
                         1375.00,1625.00,1875.00,2125.00,2375.00,2625.00,2875.00,
                         3125.00,3375.00,3625.00,3750.00])

In [ ]:
spend1.drop(columns = spend1.filter(regex = 'sc_education_', axis= 1), inplace= True)
spend1.drop(columns = spend1.filter(regex = 'sc_travel_services_', axis= 1), inplace= True)

### Keep the highest categories of *spend*

In [ ]:
overall = spend1.filter(regex='overall',axis=1)
automotive = spend1.filter(regex = 'c_automotive',axis= 1)
communication = spend1.filter(regex = 'c_communication',axis= 1)
education = spend1.filter(regex = 'c_education',axis= 1)
entertainment = spend1.filter(regex = 'c_entertainment',axis= 1)
financial_services = spend1.filter(regex = 'c_financial_services',axis= 1)
food_convenience_drug_store = spend1.filter(regex = 'c_food-convenience-drug_store',axis= 1)
other_retail = spend1.filter(regex = 'c_other_retail',axis= 1)
restaurant = spend1.filter(regex = "c_restaurant", axis=1)
retail = spend1.filter(regex = "c_retail", axis=1)
services = spend1.filter(regex = "c_services", axis=1)
travel = spend1.filter(regex = "c_travel", axis=1)

### Drop columns starting with *sc_* (not the highest category but not yet filtered out)

In [ ]:
education.drop(columns = education.filter(regex = 'sc_education_', axis= 1), inplace= True)
travel.drop(columns = travel.filter(regex = 'sc_travel_services_', axis= 1), inplace= True)

### Filling in missing values in each category of *spend* using iterative imputer

In [ ]:
overall_clean=pd.DataFrame(IterativeImputer(initial_strategy='median',
                                         imputation_order='ascending').fit_transform(overall),columns=overall.columns)

In [ ]:
auto_clean=pd.DataFrame(IterativeImputer(initial_strategy='median',
                                         imputation_order='ascending').fit_transform(automotive),columns=automotive.columns)

In [ ]:
comm_clean=pd.DataFrame(IterativeImputer(initial_strategy='median',
                                         imputation_order='ascending').fit_transform(communication),columns=communication.columns)

In [ ]:
educ_clean=pd.DataFrame(IterativeImputer(initial_strategy='median',
                                         imputation_order='ascending').fit_transform(education),columns=education.columns)

In [ ]:
rec_clean=pd.DataFrame(IterativeImputer(initial_strategy='median',
                                         imputation_order='ascending').fit_transform(entertainment),columns=entertainment.columns)

In [ ]:
finance_clean=pd.DataFrame(IterativeImputer(initial_strategy='median',
                                         imputation_order='ascending').fit_transform(financial_services),columns=financial_services.columns)

In [ ]:
food_clean=pd.DataFrame(IterativeImputer(initial_strategy='median',
                                         imputation_order='ascending').fit_transform(food_convenience_drug_store),columns=food_convenience_drug_store.columns)

In [ ]:
other_clean=pd.DataFrame(IterativeImputer(initial_strategy='median',
                                         imputation_order='ascending').fit_transform(other_retail),columns=other_retail.columns)

In [ ]:
restaurant_clean=pd.DataFrame(IterativeImputer(initial_strategy='median',
                                         imputation_order='ascending').fit_transform(restaurant),columns=restaurant.columns)

In [ ]:
retail_clean=pd.DataFrame(IterativeImputer(initial_strategy='median',
                                         imputation_order='ascending').fit_transform(retail),columns=retail.columns)

In [ ]:
services_clean=pd.DataFrame(IterativeImputer(initial_strategy='median',
                                         imputation_order='ascending').fit_transform(services),columns=services.columns)

In [ ]:
travel_clean=pd.DataFrame(IterativeImputer(initial_strategy='median',
                                         imputation_order='ascending').fit_transform(travel),columns=travel.columns)

### Concatenate all imputed *spend* categories into a single dataframe

In [ ]:
spend_clean=pd.concat([overall_clean, auto_clean, comm_clean, educ_clean, rec_clean, finance_clean, food_clean,
                       other_clean, restaurant_clean, retail_clean, services_clean, travel_clean], axis=1)

In [ ]:
spend_clean

,overall_0-3m_spend,overall_4-6m_spend,overall_7-9m_spend,overall_10-12m_spend,overall_13-15m_spend,overall_16-18m_spend,overall_19-21m_spend,overall_22-24m_spend,c_automotive_0-3m_spend,c_automotive_4-6m_spend,c_automotive_7-9m_spend,c_automotive_10-12m_spend,c_automotive_13-15m_spend,c_automotive_16-18m_spend,c_automotive_19-21m_spend,c_automotive_22-24m_spend,c_communications_service_providers_0-3m_spend,c_communications_service_providers_4-6m_spend,c_communications_service_providers_7-9m_spend,c_communications_service_providers_10-12m_spend,c_communications_service_providers_13-15m_spend,c_communications_service_providers_16-18m_spend,c_communications_service_providers_19-21m_spend,c_communications_service_providers_22-24m_spend,c_education_0-3m_spend,c_education_4-6m_spend,c_education_7-9m_spend,c_education_10-12m_spend,c_education_13-15m_spend,c_education_16-18m_spend,c_education_19-21m_spend,c_education_22-24m_spend,c_entertainment_0-3m_spend,c_entertainment_4-6m_spend,c_entertainment_7-9m_spend,c_entertainment_10-12m_spend,c_entertainment_13-15m_spend,c_entertainment_16-18m_spend,c_entertainment_19-21m_spend,c_entertainment_22-24m_spend,...,c_other_retail_0-3m_spend,c_other_retail_4-6m_spend,c_other_retail_7-9m_spend,c_other_retail_10-12m_spend,c_other_retail_13-15m_spend,c_other_retail_16-18m_spend,c_other_retail_19-21m_spend,c_other_retail_22-24m_spend,c_restaurant_0-3m_spend,c_restaurant_4-6m_spend,c_restaurant_7-9m_spend,c_restaurant_10-12m_spend,c_restaurant_13-15m_spend,c_restaurant_16-18m_spend,c_restaurant_19-21m_spend,c_restaurant_22-24m_spend,c_retail_0-3m_spend,c_retail_4-6m_spend,c_retail_7-9m_spend,c_retail_10-12m_spend,c_retail_13-15m_spend,c_retail_16-18m_spend,c_retail_19-21m_spend,c_retail_22-24m_spend,c_services_0-3m_spend,c_services_4-6m_spend,c_services_7-9m_spend,c_services_10-12m_spend,c_services_13-15m_spend,c_services_16-18m_spend,c_services_19-21m_spend,c_services_22-24m_spend,c_travel_0-3m_spend,c_travel_4-6m_spend,c_travel_7-9m_spend,c_travel_10-12m_spend,c_travel_13-15m_spend,c_travel_16-18m_spend,c_travel_19-21m_spend,c_travel_22-24m_spend
0,3750.0,1125.0,2125.0,3750.0,975.0,1875.0,3375.0,2125.0,88.000000,475.000000,112.500000,88.000000,88.000000,25.500000,62.500000,25.500000,975.000000,62.500000,275.000000,625.000000,275.000000,225.000000,88.000000,326.287420,2.062587,0.351544,1.938521,4.777622,7.945678,1.1243,12.176386,13.447436,3125.00000,62.500000,25.500000,325.000000,62.500000,62.500000,1375.000000,162.500000,...,62.5,225.000000,475.0,325.000000,25.500000,189.607557,325.0,194.517151,88.0,62.5,137.5,88.00000,62.5,88.0,112.5,25.5,825.0,137.5,575.0,1875.0,225.0,1125.0,875.0,430.0,142.599398,25.500000,158.714281,143.538086,141.448075,149.193800,160.343391,154.802553,275.000000,312.391819,25.500000,307.487486,295.143381,286.088808,275.000000,25.500000
1,3750.0,3750.0,3375.0,3750.0,3750.0,3750.0,3750.0,3750.0,112.500000,236.254340,430.000000,251.362022,225.000000,235.822346,88.000000,252.454080,430.000000,187.500000,775.000000,575.000000,137.500000,225.000000,575.000000,225.000000,2.062587,0.351544,1.938521,4.777622,7.945678,1.1243,12.176386,13.447436,25.50000,62.500000,25.500000,62.500000,25.500000,275.000000,187.500000,112.500000,...,475.0,925.000000,525.0,525.000000,430.000000,775.000000,475.0,1125.000000,875.0,775.0,430.0,375.00000,575.0,275.0,575.0,675.0,1375.0,575.0,875.0,2125.0,1125.0,1125.0,2125.0,225.0,148.096674,140.304524,158.770063,145.093707,137.500000,142.911324,25.500000,147.546143,375.000000,925.000000,225.000000,475.000000,25.500000,137.500000,775.000000,875.000000
2,1125.0,1125.0,375.0,1125.0,3750.0,775.0,1375.0,825.0,363.091249,329.347047,345.223283,343.972060,320.173306,314.261142,354.779573,317.931723,390.245078,390.340356,399.760349,398.196669,367.273375,353.589285,377.182218,332.042871,2.062587,0.351544,1.938521,4.777622,7.945678,1.1243,12.176386,13.447436,234.97624,188.392936,183.469211,180.666856,139.400782,221.796133,247.725940,250.955865,...,25.5,230.451263,225.0,2

### Merge all cleaned dataframe into a new one

In [ ]:
# cleaned data from oringial Eplison#1 and Eplison #2
df=pd.concat([count,geo,x_clean,x2_clean,dummies,spend_clean],axis=1)
df.head()

,Person Sequence Number - Person 1,city,state,zip,zip_plus4,Number of Children (Enhanced),Advantage Target Income 3.0,Discretionary Spending Income,Target Net Worth 4.0 Code,Family composition (Enhanced),Advantage Individual Marital Status - Person 1,Gender code - Person 1,Advantage Dwelling Type,Advantage Home Owner,Advantage Length of Residence,NICHES 5.0 Code,NICHES 5.0 Code (Specified),Children (categorical),Family Composition (categorical),Marital Status,Gender (categorical),Dwelling Type (categorical),Home Owner (categorical),Length of Residence (categorical),overall_0-3m_spend,overall_4-6m_spend,overall_7-9m_spend,overall_10-12m_spend,overall_13-15m_spend,overall_16-18m_spend,overall_19-21m_spend,overall_22-24m_spend,c_automotive_0-3m_spend,c_automotive_4-6m_spend,c_automotive_7-9m_spend,c_automotive_10-12m_spend,c_automotive_13-15m_spend,c_automotive_16-18m_spend,c_automotive_19-21m_spend,c_automotive_22-24m_spend,...,c_other_retail_0-3m_spend,c_other_retail_4-6m_spend,c_other_retail_7-9m_spend,c_other_retail_10-12m_spend,c_other_retail_13-15m_spend,c_other_retail_16-18m_spend,c_other_retail_19-21m_spend,c_other_retail_22-24m_spend,c_restaurant_0-3m_spend,c_restaurant_4-6m_spend,c_restaurant_7-9m_spend,c_restaurant_10-12m_spend,c_restaurant_13-15m_spend,c_restaurant_16-18m_spend,c_restaurant_19-21m_spend,c_restaurant_22-24m_spend,c_retail_0-3m_spend,c_retail_4-6m_spend,c_retail_7-9m_spend,c_retail_10-12m_spend,c_retail_13-15m_spend,c_retail_16-18m_spend,c_retail_19-21m_spend,c_retail_22-24m_spend,c_services_0-3m_spend,c_services_4-6m_spend,c_services_7-9m_spend,c_services_10-12m_spend,c_services_13-15m_spend,c_services_16-18m_spend,c_services_19-21m_spend,c_services_22-24m_spend,c_travel_0-3m_spend,c_travel_4-6m_spend,c_travel_7-9m_spend,c_travel_10-12m_spend,c_travel_13-15m_spend,c_travel_16-18m_spend,c_travel_19-21m_spend,c_travel_22-24m_spend
0,3300572A0311940000A51A,ALEXANDRIA,VA,22314,1747,2.0,7.0,4.0,7.0,6.0,0.0,1.0,5.0,4.0,8.0,14.0,14: Nice and Easy Living,Two,"Male householder with one or more persons, wit...",Single,Male,Condo,Definite owner,20+ years,3750.0,1125.0,2125.0,3750.0,975.0,1875.0,3375.0,2125.0,88.000000,475.000000,112.500000,88.000000,88.000000,25.500000,62.500000,25.500000,...,62.5,225.000000,475.0,325.000000,25.500000,189.607557,325.0,194.517151,88.0,62.5,137.5,88.00000,62.5,88.0,112.5,25.5,825.0,137.5,575.0,1875.0,225.0,1125.0,875.0,430.0,142.599398,25.500000,158.714281,143.538086,141.448075,149.193800,160.343391,154.802553,275.000000,312.391819,25.500000,307.487486,295.143381,286.088808,275.000000,25.50000
1,3300572A0311FA019775B8,ALEXANDRIA,VA,22314,1768,2.0,12.0,10.0,10.0,12.0,0.0,1.0,1.0,4.0,5.0,9.0,9: IRA Spenders,Two,"One person, male householder, with no children",Single,Male,SFDU,Definite owner,6-10 years,3750.0,3750.0,3375.0,3750.0,3750.0,3750.0,3750.0,3750.0,112.500000,236.254340,430.000000,251.362022,225.000000,235.822346,88.000000,252.454080,...,475.0,925.000000,525.0,525.000000,430.000000,775.000000,475.0,1125.000000,875.0,775.0,430.0,375.00000,575.0,275.0,575.0,675.0,1375.0,575.0,875.0,2125.0,1125.0,1125.0,2125.0,225.0,148.096674,140.304524,158.770063,145.093707,137.500000,142.911324,25.500000,147.546143,375.000000,925.000000,225.000000,475.000000,25.500000,137.500000,775.000000,875.00000
2,330055F3030E0C000B3592,ALEXANDRIA,VA,22314,1771,3.0,13.0,9.0,11.0,2.0,1.0,0.0,5.0,4.0,4.0,3.0,3: Chic Society,Three,"Married, husband and wife present, with no chi...",Married,Female,Condo,Definite owner,3-5 years,1125.0,1125.0,375.0,1125.0,3750.0,775.0,1375.0,825.0,363.091249,329.347047,345.223283,343.972060,320.173306,314.261142,354.779573,317.931723,...,25.5,230.451263,225.0,225.696896,184.937705,25.500000,225.0,225.000000,625.0,25.5,25.5,129.55222,25.5,112.5,112.5,137.5,475.0,975.0,137.5,975.0,3750.0,625.0,975.0,430.0,101.139711,25.500000,25.500000,25.500000,92.966958,25.500000,137.500000,88.000000,862.204518,803.457501,735.206264,785.609550,660.852053,903.451897,985.790675,1033.60495
3,3300

Generate sum_category for each category's spending over all of the time period

In [ ]:
overall_sum = overall.apply(lambda x: pd.to_numeric(x, errors='coerce').sum(), axis=1)
df["sum_overall"] = pd.DataFrame(overall_sum)

In [ ]:
automotive_sum = automotive.apply(lambda x: pd.to_numeric(x, errors='coerce').sum(), axis=1)
df["sum_automotive"] = pd.DataFrame(automotive_sum)

In [ ]:
communication_sum= communication.apply(lambda x: pd.to_numeric(x, errors='coerce').sum(), axis=1)
df["sum_communication"] = pd.DataFrame(communication_sum)

In [ ]:
education_sum= education.apply(lambda x: pd.to_numeric(x, errors='coerce').sum(), axis=1)
df["sum_education"] = pd.DataFrame(education_sum)

In [ ]:
entertainment_sum= entertainment.apply(lambda x: pd.to_numeric(x, errors='coerce').sum(), axis=1)
df["sum_entertainment"] = pd.DataFrame(entertainment_sum)

In [ ]:
financial_services_sum=financial_services.apply(lambda x: pd.to_numeric(x, errors='coerce').sum(), axis=1)
df["sum_financial_services"] = pd.DataFrame(financial_services_sum)

In [ ]:
food_convenience_drug_store_sum=food_convenience_drug_store.apply(lambda x: pd.to_numeric(x, errors='coerce').sum(), axis=1)
df["sum_food_convenience_drug_store"] = pd.DataFrame(food_convenience_drug_store_sum)

In [ ]:
other_retail_sum=other_retail.apply(lambda x: pd.to_numeric(x, errors='coerce').sum(), axis=1)
df["sum_other_retail"] = pd.DataFrame(other_retail_sum)

In [ ]:
restaurant_sum=restaurant.apply(lambda x: pd.to_numeric(x, errors='coerce').sum(), axis=1)
df["sum_restaurant"]= pd.DataFrame(restaurant_sum)

In [ ]:
retail_sum=retail.apply(lambda x: pd.to_numeric(x, errors='coerce').sum(), axis=1)
df["sum_retail"]= pd.DataFrame(retail_sum)

In [ ]:
services_sum=services.apply(lambda x: pd.to_numeric(x, errors='coerce').sum(), axis=1)
df["sum_services"]= pd.DataFrame(services_sum)

In [ ]:
travel_sum= travel.apply(lambda x: pd.to_numeric(x, errors='coerce').sum(), axis=1)
df["sum_travel"]= pd.DataFrame(travel_sum)

Remove time series within each category

In [ ]:
df.drop(columns = df.filter(regex = 'overall_', axis=1), inplace = True)
df.drop(columns = df.filter(regex = 'c_automotive', axis= 1), inplace= True)
df.drop(columns = df.filter(regex = 'c_communication', axis= 1), inplace= True)
df.drop(columns = df.filter(regex = 'c_education', axis= 1), inplace= True)
df.drop(columns = df.filter(regex = 'c_entertainment', axis= 1), inplace= True)
df.drop(columns = df.filter(regex = 'c_financial_services', axis= 1), inplace= True)
df.drop(columns = df.filter(regex = 'c_food-convenience-drug_store', axis= 1), inplace= True)
df.drop(columns = df.filter(regex = 'c_other_retail', axis= 1), inplace= True)
df.drop(columns = df.filter(regex = 'c_restaurant', axis= 1), inplace= True)
df.drop(columns = df.filter(regex = 'c_retail', axis= 1), inplace= True)
df.drop(columns = df.filter(regex = 'c_services', axis= 1), inplace= True)
df.drop(columns = df.filter(regex = 'c_travel', axis= 1), inplace= True)

In [ ]:
df

,Person Sequence Number - Person 1,city,state,zip,zip_plus4,Number of Children (Enhanced),Advantage Target Income 3.0,Discretionary Spending Income,Target Net Worth 4.0 Code,Family composition (Enhanced),Advantage Individual Marital Status - Person 1,Gender code - Person 1,Advantage Dwelling Type,Advantage Home Owner,Advantage Length of Residence,NICHES 5.0 Code,NICHES 5.0 Code (Specified),Children (categorical),Family Composition (categorical),Marital Status,Gender (categorical),Dwelling Type (categorical),Home Owner (categorical),Length of Residence (categorical),sum_overall,sum_automotive,sum_communication,sum_education,sum_entertainment,sum_financial_services,sum_food_convenience_drug_store,sum_other_retail,sum_restaurant,sum_retail,sum_services,sum_travel
0,3300572A0311940000A51A,ALEXANDRIA,VA,22314,1747,2.0,7.0,4.0,7.0,6.0,0.0,1.0,5.0,4.0,8.0,14.0,14: Nice and Easy Living,Two,"Male householder with one or more persons, wit...",Single,Male,Condo,Definite owner,20+ years,19100.0,965.0,2525.5,0.0,5200.5,0.0,3476.0,1438.0,664.5,6067.5,25.5,601.0
1,3300572A0311FA019775B8,ALEXANDRIA,VA,22314,1768,2.0,12.0,10.0,10.0,12.0,0.0,1.0,1.0,4.0,5.0,9.0,9: IRA Spenders,Two,"One person, male householder, with no children",Single,Male,SFDU,Definite owner,6-10 years,29625.0,855.5,3130.0,0.0,776.5,0.0,6250.0,5255.0,4555.0,9550.0,163.0,3813.0
2,330055F3030E0C000B3592,ALEXANDRIA,VA,22314,1771,3.0,13.0,9.0,11.0,2.0,1.0,0.0,5.0,4.0,4.0,3.0,3: Chic Society,Three,"Married, husband and wife present, with no chi...",Married,Female,Condo,Definite owner,3-5 years,10475.0,0.0,0.0,0.0,0.0,0.0,25.5,726.0,1064.0,8342.5,327.5,0.0
3,3300572A0310CF0001DD4F,ALEXANDRIA,VA,22314,1771,1.0,13.0,12.0,9.0,2.0,1.0,0.0,5.0,4.0,5.0,4.0,4: Doing Well and Donating,One,"Married, husband and wife present, with no chi...",Married,Female,Condo,Definite owner,6-10 years,30000.0,3162.5,1450.0,0.0,628.0,525.0,6500.0,14325.0,3592.5,18600.0,926.5,6663.0
4,3300571D0310C7000083BD,ALEXANDRIA,VA,22314,1792,1.0,6.0,4.0,3.0,12.0,0.0,1.0,5.0,4.0,5.0,10.0,10: Just Sailing Along,One,"One person, male householder, with no children",Single,Male,Condo,Definite owner,6-10 years,29875.0,563.5,3737.5,0.0,1175.0,1001.0,7600.0,9550.0,1762.5,10655.0,2426.5,5950.5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
150931,1800514F030E0F00004869,CHEVY CHASE,MD,20815,7284,2.0,13.0,10.0,11.0,14.0,1.0,0.0,5.0,4.0,7.0,9.0,9: IRA Spenders,Two,"One person, female householder, with no children",Married,Female,Condo,Definite owner,16-20 years,28375.0,877.0,2288.0,0.0,413.0,0.0,11805.0,3900.0,1263.0,6368.0,214.5,6300.0
150932,1800514F0315190060D211,CHEVY CHASE,MD,20815,7290,1.0,6.0,5.0,5.0,14.0,0.0,0.0,5.0,4.0,2.0,14.0,14: Nice and Easy Living,One,"One person, female householder, with no children",Single,Female,Condo,Definite owner,7-12 months,20875.0,326.5,726.5,0.0,1037.5,2425.0,3787.5,4517.5,1363.0,7800.0,127.5,102.0
150933,18005151030E0C0016BE69,BETHESDA,MD,20816,1013,1.0,13.0,12.0,10.0,1.0,1.0,1.0,1.0,4.0,7.0,5.0,5: Easy Street,One,"Married, husband and wife present, with children",Married,Male,SFDU,Definite owner,16-20 years,30000.0,15250.0,10100.0,0.0,9287.5,4875.0,24250.0,29375.0,5005.0,29375.0,3855.5,28875.0
150934,060167CD0311F30127ABF4,BETHESDA,MD,20816,1030,2.0,13.0,12.0,8.0,2.0,1.0,0.0,1.0,4.0,5.0,2.0,2: Big Spender Parents,Two,"Married, husband and wife present, with no chi...",Married,Female,SFDU,Definite owner,6-10 years,28375.0,76.5,2180.0,0.0,51.0,25.5,5413.5,10800.0,1337.5,11500.0,1152.0,15837.5


export to csv file

In [ ]:
from google.colab import  drive
drive.mount('/drive')
df.to_csv('/drive/My Drive/cleaned_data_ayye.csv')

Mounted at /drive
